# 对食物商品的价格分析

分析目标
- 商品的综合最低价、中位数、最高价
- 根据时间排布的价格曲线 线性时间、季节
- 哪种产品，次种
- 哪里的产品
- 同类产品不同特色的影响
- 

In [1]:
# 导包
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
%matplotlib inline
# 设置中文字体
plt.rcParams['font.sans-serif'] = 'SimHei'  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号
sns.set(style="whitegrid", font='SimHei')

In [3]:
# 读取元数据
datafile = './xinfadi_price_detail.csv'
df = pd.read_csv(datafile)


# 调整日期
def f( time ):
    return time.split(" ")[0]
  
# 数据处理部分
df.drop(columns='status', inplace=True)
df['pubDate'] = df['pubDate'].apply(f)
df['pubDate'] = pd.to_datetime(df['pubDate'])
df['place'].fillna('', inplace=True)
df['place'] = df['place'].str.replace('\\', '')
df['place'] = df['place'].apply(lambda x: ' '.join(list(x)))

/tmp/ipykernel_27362/3980508558.py:3: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(datafile)


In [4]:
df.head()

,id,prodName,prodCatid,prodCat,prodPcatid,prodPcat,lowPrice,highPrice,avgPrice,place,specInfo,unitInfo,pubDate
0,1554649,大白菜,1186,蔬菜,NaN,NaN,0.40,0.6,0.50,冀 鲁 鄂,NaN,斤,2024-01-02
1,1554648,娃娃菜,1186,蔬菜,NaN,NaN,0.70,1.0,0.85,豫,NaN,斤,2024-01-02
2,1554647,小白菜,1186,蔬菜,NaN,NaN,1.80,2.2,2.00,鲁 冀,NaN,斤,2024-01-02
3,1554646,圆白菜,1186,蔬菜,NaN,NaN,0.45,0.5,0.48,鄂 冀,露地,斤,2024-01-02
4,1554645,圆白菜,1186,蔬菜,NaN,NaN,2.00,2.4,2.20,鲁 冀,大棚,斤,2024-01-02


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579795 entries, 0 to 579794
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id          579795 non-null  int64         
 1   prodName    579795 non-null  object        
 2   prodCatid   579795 non-null  int64         
 3   prodCat     579795 non-null  object        
 4   prodPcatid  237239 non-null  float64       
 5   prodPcat    301241 non-null  object        
 6   lowPrice    579795 non-null  float64       
 7   highPrice   579795 non-null  float64       
 8   avgPrice    579795 non-null  float64       
 9   place       579795 non-null  object        
 10  specInfo    343086 non-null  object        
 11  unitInfo    579795 non-null  object        
 12  pubDate     579795 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(6)
memory usage: 57.5+ MB


In [6]:
# 抽取种类
categories = df.prodCat.unique()
categories

array(['蔬菜', '水果', '水产', '肉禽蛋', '豆制品', '调料', '粮油'], dtype=object)

In [7]:
# 构建种类dataframe

cat_df_dict = {}
for cat in categories:
    print(cat)
    cat_df_dict[cat] = df[df['prodCat'].values == cat]
cat_df_dict

蔬菜
水果
水产
肉禽蛋
豆制品
调料
粮油


{'蔬菜':              id prodName  prodCatid prodCat  prodPcatid prodPcat  lowPrice  \
 0       1554649      大白菜       1186      蔬菜         NaN      NaN      0.40   
 1       1554648      娃娃菜       1186      蔬菜         NaN      NaN      0.70   
 2       1554647      小白菜       1186      蔬菜         NaN      NaN      1.80   
 3       1554646      圆白菜       1186      蔬菜         NaN      NaN      0.45   
 4       1554645      圆白菜       1186      蔬菜         NaN      NaN      2.00   
 ...         ...      ...        ...     ...         ...      ...       ...   
 579327   970140      散生菜       1186      蔬菜         NaN      NaN      3.50   
 579328   970139      奶白菜       1186      蔬菜         NaN      NaN      2.50   
 579329   970138      油麦菜       1186      蔬菜         NaN      NaN      2.50   
 579330   970137      小白菜       1186      蔬菜         NaN      NaN      2.50   
 579331   970136       油菜       1186      蔬菜         NaN      NaN      1.50   
 
         highPrice  avgPrice  place specInfo

In [8]:
def get_price_data( name, price_dict ):
    '''获取价格字典'''
    product_data = df[df['prodName'] == name]

    unit = '斤'

    # 找到最低价、平均价和最高价所在的行的索引
    min_price_index = product_data['lowPrice'].idxmin()
    min_avg_price_index = product_data['avgPrice'].idxmin()
    max_price_index = product_data['highPrice'].idxmax()

    # 获取对应信息
    minPrice = df.loc[min_price_index, 'lowPrice']
    minAvgPrice = df.loc[min_avg_price_index, 'avgPrice']
    maxPrice = df.loc[max_price_index, 'highPrice']

    # 添加到字典中
    price_dict['prodName'].append(name)
    price_dict['minPrice'].append(minPrice)
    price_dict['minAvgPrice'].append(minAvgPrice)
    price_dict['maxPrice'].append(maxPrice)

    return (unit, price_dict)

In [9]:
def price_plot_func( price_df, unit, cat ):
    '''绘制价格bar,保存图像'''
    # 绘制最低价、平均价和最高价的条形图
    plt.figure(figsize=(100, 10), dpi=300)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    sns.barplot(x='prodName', y='minPrice', data=price_df, color='blue', label='Min Price')
    plt.legend()
    plt.title(f"{cat}最低价统计 (元/{unit})")
    plt.savefig(f"./imgs/price_da/高低价统计/{cat}最低价统计.png")

    plt.figure(figsize=(100, 10), dpi=300)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    sns.barplot(x='prodName', y='minAvgPrice', data=price_df, color='green', label='Min Avg Price')
    plt.legend()
    plt.title(f"{cat}最低均价统计 (元/{unit})")
    plt.savefig(f"./imgs/price_da/高低价统计/{cat}最低均价统计.png")

    plt.figure(figsize=(100, 10), dpi=300)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    sns.barplot(x='prodName', y='maxPrice', data=price_df, color='red', label='Max Price')
    plt.legend()
    plt.title(f"{cat}最高价统计 (元/{unit})")
    plt.savefig(f"./imgs/price_da/高低价统计/{cat}最高价统计.png")

In [10]:
def price_analyze_one( df, cat ):
    '''按类别分析最低价、最低均价、最高价'''
    product_names = df['prodName'].unique()

    # 创建一个空字典，用于存储每个产品的统计信息
    price_dict = {'prodName'   : [],
                  'minPrice'   : [],
                  'minAvgPrice': [],
                  'maxPrice'   : []}

    unit = ''

    # 遍历每个产品
    for name in product_names:
        unit, price_dict = get_price_data(name, price_dict)

        # 将字典转换为DataFrame
    price_df = pd.DataFrame(price_dict)
    print(price_df['prodName'].tolist())

    price_plot_func(price_df, unit, cat)

In [11]:
def price_analyze_two( df, cat ):
    '''分析价格分布,价格前10,后10'''
    avg_price_prod_df = df.groupby('prodName')['avgPrice'].mean()
    unit = '斤'

    bins = [0, 10, 20, 40, 1000]
    categories = pd.cut(avg_price_prod_df, bins=bins, labels=['0-10', '10-20', '20-40', '>40'])
    
    # 获取价格分布
    category_distribution = categories.value_counts(normalize=True)
    
    # 作图
    plt.figure(figsize=(8, 8))
    plt.pie(category_distribution, labels=category_distribution.index, autopct='%1.1f%%', startangle=90,
            pctdistance=0.8, wedgeprops=dict(width=0.9), explode=[0.1, 0, 0, 0])
    plt.title(f'{cat}价格分布 元/{unit}')
    plt.savefig(f'./imgs/price_da/均价分析/{cat}价格分布.png')

    # 获取均价排行
    avg_price_prod_df.sort_values(ascending=False, inplace=True)
    top_ten = avg_price_prod_df.head(10)
    bottom_ten = avg_price_prod_df.tail(10)

    # 作图
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    sns.barplot(x=top_ten.index, y=top_ten.values, palette='viridis')
    plt.xticks(rotation=45, ha='right')
    plt.title(f'{cat}高价TOP10 元/{unit}')

    plt.subplot(1, 2, 2)
    sns.barplot(x=bottom_ten.index, y=bottom_ten.values, palette='viridis')
    plt.xticks(rotation=45, ha='right')
    plt.title(f'{cat}低价TOP10 元/{unit}')

    plt.tight_layout()
    plt.savefig(f"./imgs/price_da/均价分析/{cat}均价统计.png")

In [12]:
def price_analyze_three( df, cat ):
    '''价格曲线 '''
    products = random.sample(df['prodName'].unique().tolist(), 10)
    plt.figure(figsize=(15, 15))
    for product in products:
        # 筛选特定商品的数据
        product_data = df[df['prodName'] == product]

        # 按日期排序
        product_data = product_data.sort_values(by='pubDate')

        # 绘制价格曲线
        sns.lineplot(x=product_data['pubDate'], y=product_data['avgPrice'], label=product)
    # 添加标签和标题
    plt.xlabel('日期')
    plt.ylabel('均价')
    plt.title(f'{cat}价格曲线(部分)')
    plt.ylim(0, 20)
    plt.xticks(pd.date_range(start=df['pubDate'].min(), end=df['pubDate'].max(), freq='3M'), rotation=45)

    # 自动调整日期标签
    plt.gcf().autofmt_xdate()

    # 显示图形
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.savefig(f'./imgs/price_curve/{cat}价格曲线.png')

In [13]:
def area_analyze( df, cat ):
    # 统计每个地区的产品数量
    place_counts = df['place'].str.split(expand=True).stack().value_counts()
    places = place_counts.keys().tolist()
    
    # 计算每个地区的占比
    place_percentage = place_counts / place_counts.sum()

    # 排序以便更好地可视化
    place_percentage = place_percentage.sort_values(ascending=False)

    # 绘制水平条形图
    plt.figure(figsize=(12, 8))
    sns.barplot(x=place_percentage.values, y=place_percentage.index, color='skyblue')
    plt.xlabel('占比')
    plt.ylabel('地区')
    plt.title('每个地区的占比')
    plt.savefig(f'./imgs/place_da/{cat}地区占比')
    
    # 构建价格地区字典
    price_place_dict = {}
    for p in places:
        place_avg_price = df[df['place'].isin([p])]['avgPrice'].mean()
        if pd.notna(place_avg_price):
            price_place_dict[p] = place_avg_price
    
    # 将字典转换为DataFrame
    df_plot = pd.DataFrame(list(price_place_dict.items()), columns=['地区', '平均价格'])
    
    # 绘制散点图
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x='地区', y='平均价格', data=df_plot, color='skyblue')
    plt.xlabel('地区')
    plt.ylabel('平均价格')
    plt.title(f'地区与{cat}价格关系')
    plt.savefig(f'./imgs/place_da/地区与{cat}价格关系')


In [ ]:
# 启动分析任务
for cat in categories:
    price_analyze_one(cat_df_dict[cat],cat)
    price_analyze_two(cat_df_dict[cat],cat)
    price_analyze_three(cat_df_dict[cat], cat)
    area_analyze(cat_df_dict[cat], cat)